<a href="https://colab.research.google.com/github/saheedniyi02/fpl-ai/blob/master/clean_previous_seasons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
#from config import *
#from utils import get_all_players_last_stats,create_features
import warnings
warnings.filterwarnings('ignore')

In [ ]:
seasons=["2020-21","2021-22","2022-23"]

unused_columns=["xP","opponent_team","expected_assists","expected_goal_involvements","expected_goals",
                "expected_goals_conceded","element_type_ex","team_h_score","team_a_score","element" , "round" ,"fixture","starts" ]

#stats that you want to use their values for the last few games to predict a

no_last_stats=3 #number of last gameweek stats to use for prediction

history_stats=["assists","bonus","bps","clean_sheets","creativity","goals_conceded","goals_scored","ict_index","influence","minutes",
               "own_goals","penalties_missed","penalties_saved","red_cards","saves","selected","threat","total_points","transfers_in","transfers_out","value","match_result","team Goal scored","team Goal conceded"]

last_features=[f"last {no_last_stats} {stat}" for stat in history_stats]

mean_features=["assists","bonus","bps","clean_sheets","creativity","goals_conceded",
     "goals_scored","ict_index","influence","minutes"
     ,"own_goals","penalties_missed","penalties_saved",
     "red_cards","saves","selected","threat","total_points"
     ,"transfers_in","transfers_out","value","match_result","team Goal scored","team Goal conceded"]

std_features=["bps","creativity","ict_index","influence","minutes","selected","threat","total_points","transfers_in","transfers_out","value"]



dropped_columns = [
    "season",
    "opponent",
    "match_result",
    "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards"
]  # "value",



leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage





team_features=["team Goal scored","team Goal conceded"]

In [ ]:
def get_last_stats(data, stat, name, no_last_stats=3):
    name_df = data[data["name"] == name]
    seasons = ["2020-21", "2021-22", "2022-23"]
    name_df_dict = {}
    for season in seasons:
        name_df_season = name_df[name_df["season"] == season]
        list_stats = []
        stats_x = []
        for value in name_df_season[stat]:
            if len(stats_x) < no_last_stats:
                list_stats.append(np.array(stats_x))
                stats_x.append(value)
            else:
                list_stats.append(np.array(stats_x))
                stats_x = deque_and_queue(stats_x, value)
        name_df_season[f"last {no_last_stats} {stat}"] = list_stats
        name_df_dict[season] = name_df_season
    return pd.concat(name_df_dict.values())


def get_all_players_last_stats(data, stat, no_last_stats=3):
    players_df = []
    for player in data["name"].unique():
        data_player = get_last_stats(data, stat, player, no_last_stats)
        players_df.append(data_player)
    # print(pd.concat(players_df))
    return pd.concat(players_df)



def remove_neg(val):
    if val > 0:
        return val
    else:
        return -val


def deque_and_queue(stats, value):
    # if -1 in stats:
    #  return stats
    # deque
    stats = stats[1:]
    stats.append(value)
    return stats


def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0


def find_mode(vals):
    """find the mode of vals"""
    try:
        if -1 in vals:
            return -1
        return stats.mode(vals)[0][0]
    except IndexError:
        return np.nan


def find_mean(vals):
    """find the mean of vals"""
    try:
        if -1 in vals:
            return -1
        return np.mean(vals)
    except:
        return np.nan


def find_max(vals):
    """find the maximum of vals"""
    try:
        if -1 in vals:
            return -1
        return np.max(vals)
    except:
        return np.nan


def find_std(vals):
    """find the standard deviation of vals"""
    try:
        if -1 in vals:
            return -1
        return np.std(vals)
    except:
        return np.nan


def find_value_count(vals, to_count):
    """find the number of times to_count appears in vals"""
    try:
        if -1 in vals:
            return -1
        values, count = np.unique(vals, return_counts=True)
        index = np.where(values == to_count)[0][0]
        return count[index]

    except:
        return -2

def get_all_stats(data, stat, name):
    name_df = data[data["name"] == name]
    seasons = ["2020-21", "2021-22", "2022-23"]
    name_df_dict = {}
    for season in seasons:
        name_df_season = name_df[name_df["season"] == season]
        list_stats = []
        stats_x = []
        for value in name_df_season[stat]:
          list_stats.append(np.array(stats_x))
          stats_x.append(value)


        name_df_season[f"all {stat}"] = list_stats
        name_df_dict[season] = name_df_season
    return pd.concat(name_df_dict.values())


def get_all_players_all_stats(data, stat):
    players_df = []
    for player in data["name"].unique():
        data_player =  get_all_stats(data, stat, player)
        players_df.append(data_player)
    # print(pd.concat(players_df))
    return pd.concat(players_df)

def create_features(data, mean_features, std_features):
    for mean_feature in mean_features:
        print(mean_feature)
        data[f"mean {mean_feature} {no_last_stats}"] = [
            find_mean(values) for values in data[f"last {no_last_stats} {mean_feature}"]
        ]
        data[f"mean {mean_feature} all"] = [
            find_mean(values) for values in data[f"all {mean_feature}"]
        ]
        #data[f"mean {mean_feature} all"]= data[f"mean {mean_feature} all"]/data["GW"]
    for std_feature in std_features:
        print(std_feature)
        data[f"std {std_feature} {no_last_stats}"] = [
            find_std(values) for values in data[f"last {no_last_stats} {std_feature}"]
        ]
    return data



In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Fplpredict /datasets/previous_seasons.csv", index_col=0)
data_home=data[data["was_home"]==True]
data_away=data[data["was_home"]==False]
data_home["team Goal scored"]=data_home["team_h_score"]
data_home["team Goal conceded"]=data_home["team_a_score"]
data_away["team Goal conceded"]=data_away["team_h_score"]
data_away["team Goal scored"]=data_away["team_a_score"]
data=pd.concat([data_home,data_away])
#data["Goal_difference"]=data["team_h_score"]-data["team_a_score"]
data.drop(unused_columns,axis=1,inplace=True)

data["kickoff_time"]=pd.to_datetime(data["kickoff_time"])
print(data.shape)

data.drop_duplicates(["name","GW","season","team","opponent"],inplace=True)
print(data.shape)
data.sort_values(by="kickoff_time",inplace=True)


for stat in history_stats:
    print(stat)
    data=get_all_players_last_stats(data,stat)
    data=get_all_players_all_stats(data,stat)

data=create_features(data,mean_features,std_features)

data.drop([f"last {no_last_stats} {stat}" for stat in history_stats],axis=1,inplace=True)
data.drop([f"all {stat}" for stat in history_stats],axis=1,inplace=True)
data["ratio_goal_scored 3"]=((data["mean goals_scored 3"]/data["mean team Goal scored 3"]).replace(np.inf, 0))*100
data["ratio_goal_scored all"]=((data["mean goals_scored all"]/data["mean team Goal scored all"]).replace(np.inf, 0))*100

(76469, 54)
(76317, 54)
assists
bonus
bps
clean_sheets
creativity
goals_conceded
goals_scored
ict_index
influence
minutes
own_goals
penalties_missed
penalties_saved
red_cards
saves
selected
threat
total_points
transfers_in
transfers_out
value
match_result
team Goal scored
team Goal conceded
assists
bonus
bps
clean_sheets
creativity
goals_conceded
goals_scored
ict_index
influence
minutes
own_goals
penalties_missed
penalties_saved
red_cards
saves
selected
threat
total_points
transfers_in
transfers_out
value
match_result
team Goal scored
team Goal conceded
bps
creativity
ict_index
influence
minutes
selected
threat
total_points
transfers_in
transfers_out
value


In [ ]:
data["opponent"]

396          Arsenal
503            Leeds
518      Aston Villa
524           Wolves
543    Sheffield Utd
           ...      
39           Chelsea
40          Brighton
33         Brentford
762            Leeds
530        Liverpool
Name: opponent, Length: 76317, dtype: object

In [ ]:
#add opponent stats for gameweeks
list_data_gw=[]
for season in seasons:
  for GW in range(1,39):

    data_gw=data[(data["season"]==season) & (data["GW"]==GW)]
    data_opponent=data_gw.drop_duplicates(subset=["kickoff_time","team"],keep="first")[['team','mean team Goal scored 3','mean team Goal conceded 3','mean team Goal scored all','mean team Goal conceded all',"mean match_result all"]]
    #print(data_gw.shape)
    data_opponent.columns=['opponent','opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]
    #print(data_opponent.shape)
    data_gw=pd.merge(data_gw,data_opponent,on="opponent",how="left")
    list_data_gw.append(data_gw)

In [ ]:
list(data.columns)

['name',
 'position',
 'team',
 'assists',
 'bonus',
 'bps',
 'clean_sheets',
 'creativity',
 'goals_conceded',
 'goals_scored',
 'ict_index',
 'influence',
 'kickoff_time',
 'minutes',
 'own_goals',
 'penalties_missed',
 'penalties_saved',
 'red_cards',
 'saves',
 'selected',
 'threat',
 'total_points',
 'transfers_balance',
 'transfers_in',
 'transfers_out',
 'value',
 'was_home',
 'yellow_cards',
 'last_season_position',
 'percent_value',
 'position rank',
 'match_result',
 'goals_scored_ex',
 'assists_ex',
 'total_points_ex',
 'minutes_ex',
 'goals_conceded_ex',
 'creativity_ex',
 'influence_ex',
 'threat_ex',
 'bonus_ex',
 'bps_ex',
 'ict_index_ex',
 'clean_sheets_ex',
 'red_cards_ex',
 'yellow_cards_ex',
 'selected_by_percent_ex',
 'now_cost_ex',
 'season',
 'GW',
 'opponent',
 'opponent_last_season_position',
 'team Goal scored',
 'team Goal conceded',
 'mean assists 3',
 'mean assists all',
 'mean bonus 3',
 'mean bonus all',
 'mean bps 3',
 'mean bps all',
 'mean clean_sheets 

In [ ]:
data=pd.concat(list_data_gw)
data.to_csv("/content/drive/MyDrive/Fplpredict /cleaned_dataset/cleaned_previous_seasons.csv")

In [ ]:
data.shape

(86852, 120)

In [ ]:
data[pd.isna(data["opp mean team Goal scored all"])]["GW"].value_counts()

1     1552
2       80
21      67
9       32
5       30
20      29
6       28
Name: GW, dtype: int64